In [4]:
import pandas as pd
import openai
import json
import os

# set up your OpenAI API key
file_path = r"C:\Users\Amit\Google Drive\openai_api_key.txt"
with open(file_path, 'r') as file:
    # Reads the content of the file
    file_content = file.read()
openai.api_key = file_content

In [9]:
# take an inquiry and return three different phrasing.
def transform_inquiry(inquiry):
    try:
        response = openai.Completion.create(
            engine="davinci",
            prompt=f'Please take the following inquiry and convert it to three different paragraphs:\n'
                   f'1. Highly legalistic and formal phrasing, as would be used by a trained lawyer.\n'
                   f'2. High-quality phrasing, as would be used by an educated person without a legal background.\n'
                   f'3. Low-quality phrasing, as would be common in online searches.\n\n'
                   f'Input: {inquiry}\n',
            temperature=0,
            max_tokens=500
        )

        outputs = response.choices[0].text.strip().split('\n\n')  # Splits on double newlines
        if len(outputs) != 3:
            raise ValueError("The API response does not contain three separate paragraphs.")

        return outputs

    except Exception as e:
        print(f'An error occurred: {e}')
        # Handle the error by returning None or re-raising the error
        return [None, None, None]


# function for legalbench - learned hands
def check_legal_issues(text):
    response = openai.Completion.create(
        engine="davinci",
        prompt=f'answer only yes or no. Does the question implicate legal issues related to public benefits? \nInput: {text}',
        temperature=0.3,
        max_tokens=100
    )
    
    answers = response.choices[0].text.strip().split('\n')
    return answers

def main():
    # Load the CSV
    df = pd.read_csv('inquiries.csv')

    # Iterate through the inquiries and call the API
    output1_list, output2_list, output3_list = [], [], []
    for inquiry in df['inquiry']:
        output1, output2, output3 = transform_inquiry(inquiry)
        output1_list.append(output1)
        output2_list.append(output2)
        output3_list.append(output3)

    df['output1'] = output1_list
    df['output2'] = output2_list
    df['output3'] = output3_list

    # For each row and each output, check legal issues
    results = []
    for _, row in df.iterrows():
        inquiry = row['inquiry']
        data = {
            'original_inquiry': inquiry,
        }
        for i in range(1, 4):
            binary_answer, text_answer = check_legal_issues(row[f'output{i}'])
            data[f'binary_answer_output{i}'] = binary_answer
            data[f'text_answer_output{i}'] = text_answer
        results.append(data)

    with open('results.json', 'w') as file:
        json.dump(results, file, indent=4)

if __name__ == "__main__":
    main()
